In [ ]:
%run _prepare.ipynb

In [ ]:
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.plotting import figure

import param
import pandas as pd
from pandas import json_normalize

import plotly.express as px
import altair as alt
import numpy as np
import panel as pn

In [ ]:
pn.extension("plotly")
from datetime import datetime as dt
import datetime

# New Cases per Continent

In [ ]:
df = countries.groupby(["date","continent"])["new_cases"].sum().reset_index()
def display_continent(aggregation, start_date):
    plot_df = df.loc[df.date > start_date]
    plot_df = plot_df.groupby("continent").rolling(aggregation, on="date").new_cases.mean().reset_index()
    return px.line(plot_df, x="date", y="new_cases", color="continent", title="New Cases per continent")

In [ ]:
aggregation_period = pn.widgets.IntInput(value=7, start=1, end=30)
start_date = pn.widgets.DatePicker(value=(dt.today()-datetime.timedelta(days=365)).date())

In [ ]:
dashboard = pn.interact(display_continent, aggregation=aggregation_period, start_date=start_date);

In [ ]:
dashboard

In [ ]:
pn.serve(dashboard)

# Class

In [ ]:
class ContinentDashboard(param.Parameterized):
    aggregation_period = param.Integer(7, bounds=(1, 30))
    start_date = param.Date((dt.today()-datetime.timedelta(days=365)))

    @param.depends('aggregation_period', 'start_date')
    def view(self):
        plot_df = df.loc[df.date > self.start_date.date()]
        plot_df = plot_df.groupby("continent").rolling(self.aggregation_period, on="date").new_cases.mean().reset_index()
        return px.line(plot_df, x="date", y="new_cases", color="continent", title="New Cases per continent")

user = ContinentDashboard()
#pn.Column(user, user.view)

In [ ]:
pn.Row(user.param, user.view)